In [ ]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.optimizers import Adam
from keras.layers import Bidirectional
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import string
from keras.utils import to_categorical
from keras.layers import Embedding,TimeDistributed

Using TensorFlow backend.


In [ ]:
path ="/content/drive/My Drive/Colab Notebooks/all_songs_with_metadata.txt"
f=open(path)
X=[]


def clean_doc(doc):
	
	doc = doc.replace('--', ' ')
	
	tokens = doc.split()
	
	table = str.maketrans('', '', string.punctuation)
	tokens = [w.translate(table) for w in tokens]
	# remove remaining tokens that are not alphabetic
	tokens = [word for word in tokens if word.isalpha()]
	# make lower case
	tokens = [word.lower() for word in tokens]
	return tokens

tokens=clean_doc(f.read())
print(len(tokens))
length = 16
sequences = list()
for i in range(length, len(tokens)):
	# select sequence of tokens
	seq = tokens[i-length:i]
	
	line = ' '.join(seq)
	
	sequences.append(line)
lines=sequences
f.close()
print(lines[9])
print(len(lines))

182566
i do nobody in the world can get along with you you got the ways of
182550


In [ ]:
# integer encode sequences of words

tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
word_index=tokenizer.word_index
sequences = tokenizer.texts_to_sequences(lines)
sequences = np.array(sequences)
vocab_size=len(word_index)+1
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
print(sequences.shape)

seq_length = X.shape[1]
MAX_SEQUENCE_LENGTH=X.shape[1]
print(len(sequences))
print(sequences[9])
print(y.shape)
print(X.shape)
print(X[9])
#print(tokenizer.sequences_to_texts_generator([[4,23]]))

(182550, 16)
182550
[   4   58  240    7    1  122   45   67  343   23    2    2   37    1
 1224    8]
(182550, 7954)
(182550, 15)
[   4   58  240    7    1  122   45   67  343   23    2    2   37    1
 1224]


In [ ]:
embeddings_index = {}
EMBEDDING_DIM=50

f = open("/content/drive/My Drive/Colab Notebooks/glove.6B.50d.txt")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector


Found 400000 word vectors.


In [ ]:
print('Build model...')
model = Sequential()

model.add(Embedding(len(word_index)+1,EMBEDDING_DIM,weights=[embedding_matrix],input_length=MAX_SEQUENCE_LENGTH,trainable=False))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(LSTM(128,return_sequences=False))
model.add((Dense(len(word_index)+1, activation='softmax')))

optimizer = Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [ ]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    text=lines
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text))
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)
        count=0
        for i in range(150):
            
            
            x_pred=tokenizer.texts_to_sequences([sentence])[0]
            x_pred=pad_sequences([x_pred], maxlen=MAX_SEQUENCE_LENGTH, truncating='pre')
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_word=""
            for word,index in word_index.items():
              if index==next_index:
                next_word=word
                break
            count=count+1
            #next_word = sequences_to_texts_generator(next_index)

            sentence = sentence+" " + next_word+" "
            if(count>=15):
              sys.stdout.write(next_word+"\n")
              count=0
            else:
              sys.stdout.write(next_word+" ")
            sys.stdout.flush()
        print()

In [ ]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)


In [ ]:
model.fit(X, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

Epoch 1/60
182550/182550 [==============================] - 110s 604us/step - loss: 4.5622

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "to satisfy ensure you not to quit keep it in your mind and dont forget that"
to satisfy ensure you not to quit keep it in your mind and dont forget thatyou can be a man that you can be a businessman foundation of a world
stone you may be the end to be the end to you have to be
a home to be a world i cant be a businessman addict prancing in the
kitchen but the groom of a beer of the moon is a hard its a
hard its a hard its a hard its a hard rains gonna be a man
that i was a little girl who swears her to me a little girl and
i was a little girl and i was a little girl and i was a
little girl but i was a little girl and i was a little girl and
i was a little girl and i was a little girl and i was a
little girl and i was a little girl and i was a little girl and

----- diversity: 0.5
----- Generating with seed: "t

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


hard rains like this is gonna be a end to be a home like a

----- diversity: 0.5
----- Generating with seed: "she was three days old now her vengeance has been satisfied and her possessions have been"
she was three days old now her vengeance has been satisfied and her possessions have beensold so high years eve of days they took him to the turn turn turn
again she saw down in front of the morning but it aint no use to
get closer but you say you knew you will hurt but you got to stay
whyd well you know i want or it were all alone ill stand here and
youre gonna have to get down to the easy luck yes who again all within
id be grateful and theyre selling and they tell me revenge the rats can be
one day alord bring a man in the story of the hurricane of the one
we used to stand my way up and you dont one under the one i
know the truth where i was only you its just a long train before to
war on past past to scream something i want you i want you to stay

----- diversity: 1.0
----- Generating